In [1]:
import requests
from openpyxl import Workbook
from random import choice
from bs4 import BeautifulSoup
from string import ascii_lowercase
import re

In [27]:
wb = Workbook()
ws1 = wb.create_sheet('Giełda')
ws2 = wb.create_sheet('Linki')
ws3 = wb.create_sheet('Filmweb')

ws1.cell(row=1, column=1).value = 'Kod'
ws1.cell(row=1, column=2).value = 'Kurs'
ws1.cell(row=1, column=3).value = 'Zmiana procentowa'
ws1.cell(row=1, column=4).value = 'Transakcje'
row = 2

In [40]:
def generateString(n):
    return "".join(choice(ascii_lowercase) for i in range(n))
codes = [generateString(3) for _ in range(5)]
s = requests.Session()
valid_codes = []
for code in codes:
    print(code)
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    if len(soup.body.findAll('span', id='c2')) != 0:
        link_content = soup.find('font',id='f16').find('a',href=True)
        if link_content.text == 'symbol waloru':
            print("Brak kodu i podobieństwa.")
        else:
            valid_code = link_content['href'][5:]
            valid_codes.append(valid_code)
            print(f'Podobny kod to: {valid_code}')
    else:
        valid_codes.append(code)

for code in valid_codes:
    print(code)
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    price = soup.find(text="Kurs").parent.find("span").text
    percent_change = soup.find(text="Zmiana").parent.findAll("span")[1].text
    transactions = soup.find(text="Transakcje").parent.find("span").text
    print(f"{price}, {percent_change}, {transactions}")
    ws1.cell(row=row, column=1).value = code
    ws1.cell(row=row, column=2).value = price if price else 'Brak danych'
    ws1.cell(row=row, column=3).value =  percent_change if percent_change else 'Brak danych'
    ws1.cell(row=row, column=4).value =  transactions if transactions else 'Brak danych'
    row += 1

gbe
Brak kodu i podobieństwa.
pnl
Podobny kod to: pnl.uk
ohw
Brak kodu i podobieństwa.
lma
Podobny kod to: lmasfiz.pl
onz
Podobny kod to: ronzar
pnl.uk
42100.00, (+0.24%), 68
lmasfiz.pl
1145.00, (+0.42%), 1
ronzar
3.46735, (-1.77%), 


In [31]:
url = "http://www.informatyka.amw.gdynia.pl/"
request = requests.get(url)
linki = set()
beautiful_soup = BeautifulSoup(request.content, 'html.parser')
for a in beautiful_soup.find_all("a"):
    link = a.get('href')
    linki.add(link)
    print(link)
row = 1
for link in linki:
    ws2.cell(row,column=1).value = link
    row +=1

index.php
#home
#pracownie
#kadra
#student
#contact
#partnerzy
https://orcid.org/0000-0001-8398-6087
None
None
mailto:t.bodnar@amw.gdynia.pl
http://www.facebook.com/tadeusz.bodnar.3
Orcid
None
None
mailto:m.kruczko@amw.gdynia.pl
None
None
mailto:t.kutowinski@amw.gdynia.pl
https://orcid.org/0000-0001-7474-2187
None
None
mailto:t.niemczyk@amw.gdynia.pl
Orcid
None
None
mailto:j.ogrodniczak@amw.gdynia.pl
https://orcid.org/0000-0001-5716-4485
None
None
mailto:m.pabich@amw.gdynia.pl
http://www.pabich.cba.pl
https://orcid.org/0000-0003-4261-8688
None
None
mailto:p.rodwald@amw.gdynia.pl
http://www.rodwald.pl
https://www.linkedin.com/in/prodwald
http://www.facebook.com/prodwald
https://orcid.org/0000-0001-8258-0181
None
None
mailto:j.zalewski@amw.gdynia.pl
https://www.linkedin.com/in/45817a152
http://www.facebook.com/jacek.zalewski.545
https://orcid.org/0000-0002-0364-4077
None
None
mailto:a.zak@amw.gdynia.pl
https://orcid.org/0000-0002-7139-6378
None
None
mailto:j.bednarski@amw.gdynia.pl
https

In [37]:
url = "https://www.filmweb.pl/film/To%3A+Rozdzia%C5%82+2-2019-793838"
request = requests.get(url)
beautiful_soup = BeautifulSoup(request.content, 'html.parser')

filmweb = {}
filmweb['reżyser'] = beautiful_soup.find(text="reżyseria:").next_element.find("a").text
filmweb['premiera'] = beautiful_soup.find("h1", class_="filmTitle").find('span').text[1:5]
boxoffice = beautiful_soup.find(text="boxoffice:").next_element.text
filmweb['ocena'] = beautiful_soup.find("span", itemprop="ratingValue").text
filmweb['box office']  = re.sub(r'(top #\d+)', '', boxoffice)

row = 1
for klucz, wartosc in filmweb.items():
    ws3.cell(row,column=1).value = klucz
    ws3.cell(row,column=2).value = wartosc
    row += 1

In [38]:
print(filmweb)

{'reżyser': 'Andy Muschietti', 'premiera': '2019', 'ocena': ' 6,5', 'box office': '$452 458 107 '}


In [41]:
wb.save('lab4.xlsx')